In [5]:
import os
import cv2
import numpy as np
from pathlib import Path
from ultralytics import YOLO

# === CONFIG ===
model_path = "runs/segment/Lane_V1/weights/best.pt"  # Path to your trained model
image_path = "test_images/demo.jpeg"           # Path to test image
output_path = "output/demo_1.jpeg"    # Save path

# === Load model ===
model = YOLO(model_path)  # Load your YOLOv8/v11 segmentation model

# === Load image ===
img = cv2.imread(image_path)
if img is None:
    raise FileNotFoundError(f"Image not found at {image_path}")
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# === Inference ===
results = model(img_rgb)[0]  # Inference on single image

# === Alpha blending setup ===
overlay = img.copy()
alpha = 0.5  # Transparency of mask overlays
mask_color = (0, 255, 0)  # Green mask

# === Resize and blend masks ===
if results.masks is not None:
    masks = results.masks.data.cpu().numpy()  # Shape: [N, H_mask, W_mask]
    orig_h, orig_w = img.shape[:2]

    for mask in masks:
        # Resize the mask to the original image size
        mask_resized = cv2.resize((mask * 255).astype(np.uint8), (orig_w, orig_h))

        # Create a color mask
        color_mask = np.zeros_like(img, dtype=np.uint8)
        color_mask[:, :, 1] = mask_resized  # Green channel

        # Blend with original image
        overlay = cv2.addWeighted(overlay, 1, color_mask, alpha, 0)

# === Final result ===
img_result = overlay

# === Save result ===
os.makedirs(Path(output_path).parent, exist_ok=True)
cv2.imwrite(output_path, img_result)
print(f"✓ Result saved to: {output_path}")



0: 448x640 3 lanes, 4.2ms
Speed: 1.3ms preprocess, 4.2ms inference, 1.1ms postprocess per image at shape (1, 3, 448, 640)
✓ Result saved to: output/demo_1.jpeg
